In [6]:
import os 
import pandas as pd 
import numpy as np 



In [7]:
# file_name = 'Maharashtra_AE.csv'
# maharashtra_election_df = pd.read_csv(file_name) 

# file_name = 'Maharashtra_GA.csv'
# maharashtra_ge_election_df = pd.read_csv(file_name) 

file_name = 'data/AE_RAJASTHAN.csv'
election_df = pd.read_csv(file_name)

def get_parties(year,AC):
    top_3 = [1,2,3]
    # Define conditions
    condition_general = (
        (election_df['Year'] == year) &
        (election_df['Constituency_No'] == AC) &
        (~election_df['Candidate'].isin(['None of the Above', 'NOTA']))
    )

    condition_inc = (
        (election_df['Party'] == 'INC')
    )

    # Use bitwise OR to combine conditions
    filtered_election_df = election_df[
        (condition_general & election_df['Position'].isin(top_3)) |
        (condition_inc & condition_general)
    ]
    
    return filtered_election_df.sort_values(by='Position', ascending=True)['Party'].tolist()


/tmp/ipykernel_235481/116659333.py:8: DtypeWarning: Columns (13,18,19,38,39,40,41,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  election_df = pd.read_csv(file_name)


In [10]:
# def intermediate_table(df, year):
#     constituency = df.iloc[0]['Constituency']
#     year = df.iloc[0]['Year']


#     # Filter the columns that start with "col_" and include "NOTA"
#     col_columns = [col for col in df.columns if col.startswith('col_') or col == 'NOTA']

#     # Calculate the sum of each column
#     columns_order_by_position = df[col_columns].sum().sort_values(ascending=False).index.tolist()
#     # if year in [2014,2019]:
#     #     parties_ordered = maharashtra_election_df[(maharashtra_election_df['Constituency_No'] == constituency) & (maharashtra_election_df['Year'] == year)].sort_values('Position', ascending=True)['Party'].tolist()

#     if year in [2014]:
#         parties_ordered = maharashtra_ge_election_df[(maharashtra_ge_election_df['Constituency_No'] == constituency) & (maharashtra_ge_election_df['Year'] == year)].sort_values('Position', ascending=True)['Party'].tolist()

#     # Create a dictionary for renaming columns: match with parties, keep remaining original if excess
#     max_len = len(columns_order_by_position)
#     rename_dict = {columns_order_by_position[i]: parties_ordered[i] if i < len(parties_ordered) else columns_order_by_position[i] for i in range(max_len)}

#     # Rename the columns in the original dataframe
#     df.rename(columns=rename_dict, inplace=True)
#     if len(parties_ordered) > len(columns_order_by_position): 
#         parties_ordered = parties_ordered[:len(columns_order_by_position)]
#     cols = pd.Series(df.columns)
#     for dup in cols[cols.duplicated()].unique(): 
#         cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
#     df.columns = cols
    
#     # Add INC Status Column
#     max_votes_column = df[parties_ordered].idxmax(axis=1)
#     df['INC_Status'] = max_votes_column.map(lambda x: 'WON' if x == 'INC' else 'LOSS')

    
#     # All columns should be integer columns 
#     df['Polling_Station'] = df['Polling_Station'].fillna(df['SN'])

#     top_parties = parties_ordered[:3] 

#     for party in top_parties:
#         df[f'{party} Share%'] = ((df[party] / df['Total']) * 100).round(2)

#     # Prepare the columns for the final DataFrame
#     final_columns = ['SN', 'Polling_Station', 'Constituency', 'Year'] + [item for party in top_parties for item in (party, f'{party} Share%')] + ['Total','INC_Status']
#     final_df = df[final_columns]
    
#     return final_df, constituency

def intermediate_table(df, year):
    constituency = df.iloc[0]['Constituency']
    year = df.iloc[0]['Year']

    parties_ordered = get_parties(year, constituency)
    
    # Filter the columns that contains parties_ordered and col_ and 'NOTA' 
    col_columns = [col for col in df.columns if col in parties_ordered or col.startswith('col_') or col == 'NOTA']

    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique(): 
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
    df.columns = cols
    
    # Add INC Status Column
    max_votes_column = df[col_columns].idxmax(axis=1)
    df['INC_Status'] = max_votes_column.map(lambda x: 'WON' if x == 'INC' else 'LOSS')

    
    top_parties = parties_ordered[:3] 

    for party in top_parties:
        df[f'{party} Share%'] = ((df[party] / df['Total']) * 100).round(2)

    # Prepare the columns for the final DataFrame
    final_columns = ['SN', 'Constituency', 'Year'] + [item for party in top_parties for item in (party, f'{party} Share%')] + ['Total','INC_Status']
    final_df = df[final_columns]
    
    # delete rows where SN is null
    final_df = final_df[final_df['SN'].notnull()]
    
    return final_df, constituency


In [11]:

import os 


def create_intermediate_tables_and_dump_excel_files(excel_dir, output_dir, log_file_name, year):
    os.makedirs(output_dir, exist_ok=True)
    for filename in os.listdir(excel_dir):
        if filename.endswith('.xlsx'):
            try: 
                df = pd.read_excel(os.path.join(excel_dir, filename))
                intermediate_df, constituency = intermediate_table(df, year)
                output_file_path = os.path.join(output_dir, f"{constituency}.xlsx")
                intermediate_df.to_excel(output_file_path, index=False)
            except Exception as exc: 
                with open(log_file_name, 'a') as f: 
                    f.write(f"Error processing file: {filename} - {exc}\n")
                continue
                

In [12]:
# excel_dir = 'results/cleaned_election_data/MH/GE_2014'
# output_dir = 'results/intermediate_tables/MH/GE_2014'
# log_file_name = "logs/maharshtra_loksabha_election_intermediate_2014_cleaning_log.txt"
# year = 2014

# excel_dir = 'cleaned_election_data/KA/2018'
# output_dir = 'intermediate_tables/KA/2018'
# log_file_name = "logs/karnataka_assembly_election_intermediate_2018_cleaning_log.txt"
# year = 2018

# excel_dir = 'cleaned_election_data/KA/2014'
# output_dir = 'intermediate_tables/KA/2014'
# log_file_name = "logs/karnataka_assembly_election_intermediate_2014_cleaning_log.txt"
# year = 2014

excel_dir = 'results/cleaned_election_data/RA/2018'
output_dir = 'results/intermediate_tables/RA/2018'
log_file_name = "logs/rajasthan_election_intermediate_2018_log.txt"
year = 2018

create_intermediate_tables_and_dump_excel_files(excel_dir, output_dir, log_file_name, year)


/tmp/ipykernel_235481/2027755411.py:64: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  max_votes_column = df[col_columns].idxmax(axis=1)


In [9]:
# file_path = 'cleaned_election_data/KA/2014/196.xlsx'
# file_path = 'cleaned_election_data/KA/2018/187.xlsx'
file_path = 'results/cleaned_election_data/RA/2018/130.xlsx'

df = pd.read_excel(file_path)
print(df.shape)
intermediate_df, constituency = intermediate_table(df, 2018)

print(intermediate_df.shape)


(0, 18)


IndexError: single positional indexer is out-of-bounds

In [14]:
top_3 = [1,2,3]
year = 2018 
AC = 16
# Define conditions
condition_general = (
    (election_df['Year'] == year) &
    (election_df['Constituency_No'] == AC) &
    (~election_df['Candidate'].isin(['None of the Above', 'NOTA']))
)

condition_inc = (
    (election_df['Party'] == 'INC')
)

# Use bitwise OR to combine conditions
filtered_election_df = election_df[
    (condition_general & election_df['Position'].isin(top_3)) |
    (condition_inc & condition_general)
]

filtered_election_df[['Position','Candidate','Party']].head(10)

,Position,Candidate,Party
209,1,SUMIT GODARA,BJP
210,2,VIRENDRA BENIWAL,INC
211,3,PRABHUDAYAL,IND
